## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [20]:
# Step 1 - Initialization
import random

def initialize_population(population_size, chromosome_length):
    population = []
    for i in range(population_size):
        individual = list(range(0, chromosome_length))
        random.shuffle(individual)
        population.append(individual)

    return population

N = 4
population_size = 10
pop = initialize_population(population_size, N)
for ind in pop:
    print(ind)

[1, 2, 0, 3]
[3, 1, 0, 2]
[1, 2, 0, 3]
[1, 2, 0, 3]
[2, 3, 1, 0]
[0, 2, 3, 1]
[3, 1, 2, 0]
[1, 3, 0, 2]
[0, 2, 1, 3]
[1, 0, 2, 3]


In [21]:
def generate_board(board_size, min_value, max_value):
    board = []
    for i in range(board_size):
        row = []
        for j in range(board_size):
            row.append(random.randint(min_value, max_value))
        board.append(row)

    return board

In [22]:
# Step 2 - Fitness Evaluation
def fitness_score(individual, board, penalty_score):
    penalty = 0
    for i in range(len(individual)):
        for j in range(i + 1, len(individual)):
            if i - individual[i] == j - individual[j] or i + individual[i] == j + individual[j]: #verificam daca sunt pe aceeasi diagonala, luam in considerare
                #ambele diagonale, principala. daca sunt pe aceeasi, aplicam penalty. fiecare solutie (permutare) are un scor de penalitate 
                penalty -= penalty_score

    if penalty < 0:
        return penalty  #daca o sol are penalitate, returnam penalitatea, pentru ca reginele se pot ataca, si practic e cumva sansa sa se atace oricare 
                        #din ele

    score = 0
    for i in range(len(individual)):
        score += board[i][individual[i]]   #scorul 

    return score

def all_fitness_scores(population, board, penalty_score):
    scores = []
    for individual in population:  
        scores.append(fitness_score(individual, board, penalty_score))
    return scores

board = generate_board(N, 10, 100) #initialize board with weights
for row in board:
    print(row)  #print the weights of each psoition of the board
print()
penalty_score = 10  #penalizam 
f_scores = all_fitness_scores(pop, board, penalty_score)
print(f_scores)  #print score of each permutation (sol in pop)

[60, 39, 23, 99]
[16, 47, 60, 54]
[56, 20, 82, 22]
[98, 34, 17, 76]

[-10, -10, -10, -10, -20, -10, -20, 166, -20, -20]


In [14]:
# Step 3 - Selection
import numpy as np

def roulette_wheel_selection(population, f_scores, selection_size):
    min_score = min(f_scores)
    positive_scores = []
    for score in f_scores:
        positive_scores.append(score - min_score + 1)  #transform all scores into positive ones so that we can use them to compute probabilities

    total_score = sum(positive_scores)
    probabilities = [score / total_score for score in positive_scores]

    indices = list(range(len(population)))
    selected_elements = np.random.choice(indices, size=selection_size, replace=False, p=probabilities) #select parents according to probabilities 
                                                                #that are based on each individual's fitness score
    #this returns tuple with the two positions of the two parents in the population
    return selected_elements[0], selected_elements[1]

parent_indices = roulette_wheel_selection(pop, f_scores, 2)
print(parent_indices)  #parent indices is a tuple with the indexes of the chosen parents from the list of individuals in pop

(5, 1)


In [15]:
# Step 4 - Crossover
def crossover(population, parent_indices):
    parent_1 = population[parent_indices[0]]
    parent_2 = population[parent_indices[1]]

    left = random.randint(0, len(parent_1) - 1)
    right = random.randint(0, len(parent_1) - 1)
    if left > right:
        aux = left
        left = right
        right = aux

    offspring = []
    for i in range(left, right + 1):
        offspring.append(parent_1[i])

    for element in parent_2:
        if element in offspring:
            continue
        else:
            offspring.append(element)

    return offspring

print(crossover(pop, parent_indices))

[3, 2, 1, 0]


In [16]:
# Step 5 - Mutation
def swap_mutation(individual):
    if random.random() <= 0.1:
        i = random.randint(0, len(individual) - 1)
        j = random.randint(0, len(individual) - 1)

        aux = individual[i]
        individual[i] = individual[j]
        individual[j] = aux

    return individual

print(swap_mutation([2, 3, 0, 1]))

[2, 3, 0, 1]


In [17]:
# Step 6 - Replace parent with offspring
def replace(population, parent_indices, parent_scores, offspring, offspring_score):
    if parent_scores[0] < parent_scores[1] and offspring_score > parent_scores[0]:
        population[parent_indices[0]] = offspring
    elif parent_scores[1] < parent_scores[0] and offspring_score > parent_scores[1]:
        population[parent_indices[1]] = offspring
    return population

In [18]:
# Step 7 - Repeat for multiple generations
def evolution(generations, population_size, chromosome_length, board, penalty_score):
    population = initialize_population(population_size, chromosome_length)

    for i in range(generations):
        f_scores = all_fitness_scores(population, board, penalty_score)   #compute score for each individual in population(each permutation)

        parent_indices = roulette_wheel_selection(population, f_scores, 2)  
        parent_scores = [f_scores[p] for p in parent_indices]

        offspring = crossover(population, parent_indices)
        offspring = swap_mutation(offspring)
        offspring_score = fitness_score(offspring, board, penalty_score)

        population = replace(population, parent_indices, parent_scores, offspring, offspring_score)

    return population

In [19]:
# Final Step - Solve the problem
if __name__ == '__main__':
    N = 4
    board = generate_board(N, 10, 99)
    for row in board:
        print(row)
    print()

    generations = 100
    population_size = 10
    penalty_score = 10
    solutions = evolution(generations, population_size, N, board, penalty_score)
    scores = all_fitness_scores(solutions, board, penalty_score)

    best = 0
    for i in range(len(solutions)):
        if scores[i] > scores[best]:
            best = i

    print(f'Best solution: {solutions[best]} having score {scores[best]}')

[57, 98, 85, 29]
[41, 11, 75, 76]
[19, 44, 77, 26]
[93, 85, 43, 26]

Best solution: [1, 3, 0, 2] having score 236
